In [1]:
using Plots;gr()
using Optim
using DataFrames;using CSV
using Pkg
using LinearAlgebra;
using JuliaZH;JuliaZH.set_mirror("PKU")#JuliaZH.generate_startup("BFSU")
ccall((:openblas_get_num_threads64_, Base.libblas_name),Cint,())
LinearAlgebra.BLAS.set_num_threads(8)
using Base.Threads;nthreads()
using Meteor.ExactDiag;
using JLD2;using FileIO
using DifferentialEquations

In [69]:
function dm2vec(dm)
    return reshape(dm,:,1)
end

function vec2dm(vec)
    return reshape(vec,3,:)
end
function thermal_state(dims,nbar)
    c::Array{Complex{Float64}}=zeros(dims,dims)
    if nbar==0
        c[1,1]=1
    else
        for i in 1:dims
           c[i,i]= exp(-i*log(1/nbar+1))
        end
    end
    c=c/tr(c)
    return c        
end
    
function coherence_state(alpha)
    m=exp(alpha*a₊(dim)-alpha'*a₋(dim))*n0(dim)
    return  m*m'
end
    
function fock_state(dims,n)
    c::Matrix{ComplexF64}=zeros(dims,dims)
    if n<dims
        c[n+1,n+1]=1
    end
    return c
end
    
function equ_state(dims,n)
    c::Matrix{ComplexF64}=zeros(dims,dims)
    for i in 1:n
        c[i,i]=1/n    
    end
    return c
end
    
dim=10
η=0.084 
    
function a₋(n)      #n为维数
    s::Array{Complex{Float64}}=zeros(n,n)
    for i=1:n-1
        s[i,i+1]=sqrt(i)
    end
    return s
end

a₊(n)=adjoint(a₋(n));
a=a₊(dim)+a₋(dim)    
# a2=kron([0 0;1 0],exp(im*η*a))
# a3=kron([0 1;0 0],exp(-im*η*a))
a2=kron([0 0;1 0],im*η*a₊(dim))
a3=kron([0 1;0 0],-im*η*a₋(dim))   
#经过尝试发现,符合脉冲并不精准,都保留下来就很不精确
    
σ₊=[0 0;1 0]
σ₋=[0 1;0 0]
σ_x=[0 1;1 0]
σ_y=[0 -1im;1im 0]
σ_z=[1 0;0 -1]

function dm2vec(dm)
    return reshape(dm,:,1)
end

function vec2dm(vec)
    return reshape(vec,Int(sqrt(size(vec)[1])),:)
end
    
function temperature(nbar)
    return (6.626*1e-34*1e6*1)/(1.38*1e-23*log(1/nbar+1))
end
    
n=kron(I(2),a₊(dim)*a₋(dim))
en=kron([0 0;0 1],I(dim))

ν=1.
    
function H(Ω,Δ)
    unitary::Matrix{ComplexF64}=kron(I(dim*2),-im*(ν*n-Δ*en+Ω/2*a2+Ω'/2*a3))+kron(transpose(im*(ν*n-Δ*en+Ω/2*a2+Ω'/2*a3)),I(dim*2))
    return unitary
end
    
function Hv(Ω,Δ)
    unitary::Matrix{ComplexF64}=ν*n-Δ*en+Ω/2*a2+Ω'/2*a3
    return unitary
end


Hv (generic function with 1 method)

In [70]:
instate=kron([1. 0;0 0],thermal_state(dim,1));

In [71]:
function R(t,phi,omega,delta)
    return exp(-im*t*Hv(omega*exp(im*phi),delta))
end

R (generic function with 1 method)

In [257]:
fg=8
function swap_prod(dims,n)#蓝边带交换
    s=zeros(dim*2,dim*2)+I(dim*2)
    s[n+1,n+1]=0.;
    s[n+2+dims,n+2+dims]=0.;
    s[n+2+dims,n+1]=1.;
    s[n+1,n+2+dims]=1.;
    
    
    return s
end
swap=swap_prod(dim,fg)

20×20 Matrix{Float64}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

In [240]:
function feval(x)
    swapfit=R(x[6],x[11],0.1,1)*R(x[5],x[10],0.1,1)*R(x[4],x[9],0.1,1)*R(x[3],x[8],0.1,1)*R(x[2],x[7],0.1,1)*R(x[1],0,0.1,1)
    opres=norm((norm.(swapfit)-norm.(swap)))
    println(round.(x,digits=4),"    ",round(opres,digits=3))
    return opres
    
end

feval (generic function with 1 method)

In [258]:
# xinx=res.minimizer

# xini=[573.7137, 1219.785, 1456.7146, 1288.4158, 1440.3245, 642.0739, 3.5509, 5.5899, 4.236, 2.9267, 4.8421]#from0
# xini=[369.5098, 834.0317, 1085.7994, 938.9236, 1062.6676, 464.2027, 3.8722, 5.2429, 4.4087, 3.5541, 5.1155]#from1
# xini=[727.6884, 1514.5717, 966.8284, 457.8239, 1424.2311, 1454.1626, 0.0417, 4.7884, 5.0925, 2.2679, 0.6187]#from2
# xini=[684.8812, 1530.8075, 963.637, 393.4166, 1380.9979, 1483.5429, 1.3643, 3.536, 5.0499, 3.8791, 5.4344]#from3
# xini=[609.1726, 1708.4377, 1001.1728, 283.1465, 1519.6246, 1626.2544, 0.8631, 1.7695, 1.1954, 5.4596, 1.49]#from4
# xini=[663.5298, 1705.9138, 1104.5907, 348.3758, 1526.8731, 1594.5831, 0.5704, 6.0259, 3.2216, 3.938, 4.6233]#from5
# xini=[670.2339, 1670.3573, 1114.2551, 405.4513, 1408.3182, 1650.8949, 0.4176, 1.3664, 5.0648, 4.4654, 4.1644]#from6
# xini=[722.4948, 1774.6254, 1028.0052, 483.0344, 1360.6366, 1658.0471, 0.8279, 6.5419, 2.2354, 2.97, 1.8705]#from7
xini=[481.5589, 1957.4248, 1247.3075, 475.1129, 1694.4807, 1529.1695, 1.2013, 1.8026, 0.1924, 1.3006, 2.6701]#from8

lower=[0.,0,0,0,0,0,0,0,0,0,0];upper=[4*pi/η/0.1,4*pi/η/0.1,4*pi/η/0.1,4*pi/η/0.1,4*pi/η/0.1,4*pi/η/0.1,2*pi,2*pi,2*pi,2*pi,2*pi];
# res=optimize(feval,xini,ParticleSwarm(; lower = lower,upper = upper,n_particles = 22),Optim.Options(iterations = 2000))
# res=optimize(feval,xini,LBFGS())
    

In [259]:
xini

11-element Vector{Float64}:
  481.5589
 1957.4248
 1247.3075
  475.1129
 1694.4807
 1529.1695
    1.2013
    1.8026
    0.1924
    1.3006
    2.6701

In [260]:
res.minimizer

11-element Vector{Float64}:
  481.55892590332
 1957.4248236113626
 1247.3074708595832
  475.11290839621785
 1694.4807416389383
 1529.1694740406024
    1.2013450497210052
    1.802598915050952
    0.19241416056966631
    1.3005512133565453
    2.6701427898027212

In [244]:
res.minimum

0.7043674014040179

In [245]:
function feval2(x)
    swapfit=R(x[6],x[11],0.1,1)*R(x[5],x[10],0.1,1)*R(x[4],x[9],0.1,1)*R(x[3],x[8],0.1,1)*R(x[2],x[7],0.1,1)*R(x[1],0,0.1,1)
     
    return swapfit
end

feval2 (generic function with 1 method)

In [261]:
round.(DataFrame(abs.(feval2(xini)),:auto),digits=4)

Row,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.9592,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2826,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.9729,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2313,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.9941,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.108,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.9976,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0691,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.9907,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1359,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.9966,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0823,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.9892,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1463,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.993,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1179,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1863,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9825


In [250]:
round.(DataFrame(abs.(feval2(res.minimizer)),:auto),digits=4)

Row,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.9592,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2826,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.9729,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2313,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.9941,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1081,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.9976,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0691,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.9907,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1359,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.9966,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0823,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.9892,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1463,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.993,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1179,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1863,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9825
